# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from tqdm import tqdm
from tqdm import notebook

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import lightgbm as lgb

import time

In [ ]:
RANDOM_STATE = 220523

# Getting acquainted with the data

In [ ]:
try:
    arc = pd.read_csv('/Users/BrightFuture/Desktop/Projects/Final project industry/data/data_arc_new.csv')
except FileNotFoundError:
    print('Error: File not found. Please provide the correct file path.')

In [ ]:
try:
    arc = pd.read_csv('/datasets/data_arc_new.csv')
    bulk = pd.read_csv('/datasets/data_bulk_new.csv')
    bulk_time = pd.read_csv('/datasets/data_bulk_time_new.csv')
    gas = pd.read_csv('/datasets/data_gas_new.csv')
    temp = pd.read_csv('/datasets/data_temp_new.csv')
    wire = pd.read_csv('/datasets/data_wire_new.csv')
    wire_time = pd.read_csv('/datasets/data_wire_time_new.csv')
except FileNotFoundError:
    print('Error: Files not found. Please provide the correct file path.')

In [ ]:
display(arc)

In [ ]:
arc[columns] = ['key', 'heat-start', 'heat_end', 'active_power', 'reactive_power']

In [ ]:
try:
    bulk = pd.read_csv('/Users/BrightFuture/Desktop/Projects/Final project industry/data/data_bulk_new.csv')
except FileNotFoundError:
    print('Error: File not found. Please provide the correct file path.')

In [ ]:
display(bulk)

In [ ]:
bulk.hist(figsize=(12,12))

Вот где пока ничего не понятно. В контексте сталелитейного завода термин «bulk» может относиться к материалам или веществам, которые транспортируются или обрабатываются в больших количествах, из чего можно сделать предположение, что у нас в произведстве участвуют 15 различных элементов, которые поэтапоно (хотя насчет порядка не уверен) добавляются нашу расплавленную смесь, чтобы получить итоговый продукт. 

Какие продукты там добавляются мы не знаем. 

Также мы не знаем, что предстваляют собой объекты. Если в прошлой таблице это были нагревательные сессии, то сейчас я не имею исчерпывающего понимания. Следует расспросить про эту таблицу на встрече с заказчиком. 

UPd. С объектами теперь стало понятно - это произведенные партии.

Если обратить внимание на гистограммы, то можно порадоваться, что они есть;)) 

А если без шуток, то мы элементарно узнаем, что какие-то присадки добавляются чаще, какие-то реже, каких-то нужно больше, каких-то меньше? Зачем оно нам? А чтобы понять, что нам это не важно и ничего необычного тут нет. Тем более нам даже названия присадок не дают, так что мы просто обработаем эти данные и добавим в котел нашей модели. И под обработаем я имею в виду всего лишь "удалим неактуальные партии".

In [ ]:
try:
    bulk_time = pd.read_csv('/Users/BrightFuture/Desktop/Projects/Final project industry/data/data_bulk_time_new.csv')
except FileNotFoundError:
    print('Error: File not found. Please provide the correct file path.')

In [ ]:
display(bulk_time)

In [ ]:
#bulk_time.info()

In [ ]:
bulk_columns = ['Bulk 1', 'Bulk 2', 'Bulk 3', 'Bulk 4', 'Bulk 5', 'Bulk 6', 'Bulk 7', 'Bulk 8', 'Bulk 9',
                'Bulk 10', 'Bulk 11', 'Bulk 12', 'Bulk 13', 'Bulk 14', 'Bulk 15']

оК. Есть понимание, что эти 2 таблицы связаны. Получается, в таблице bulk количество, а в bulk_time время каждого добавления каждого элемента. Я был прав, что порядок не определен, но по идее это и не важно. Что еще примечательно, у нас есть таймлайн, по которому можно понять, что каждому "балку" предшествует нагревание.  

Upd. Судя по описанию, скорее всего, это легирующие добавки.  При чем не все, а те, которые подаются их бункера для сыпучих материалов. 

Пока что в голову приходит только одна идея - сделать какую-то таблицу с общим таймлайном, где будут как нагревания, так и добавления элементов. Зачем  она нужна? я пока не знаю, но лучше такая идея, чем никакой. 

In [ ]:
try:
    gas = pd.read_csv('/Users/BrightFuture/Desktop/Projects/Final project industry/data/data_gas_new.csv')
except FileNotFoundError:
    print('Error: File not found. Please provide the correct file path.')

In [ ]:
display(gas)

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(gas['Газ 1'], bins=20, color='skyblue', edgecolor='black', alpha=0.7)

plt.xlabel('Количество')
plt.ylabel('Частота')
plt.title('Обзор количества газа в производстве')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

Пока снова ничего не понятно, но хорошая новость в том, что количество объектов такое же, как и в двух предыдущих таблицах, так что можем предположить, что это количество газа, требуемое для выработки энергии. Или, может, это какой-то катализатор?  

Оказалось, что газ-это катализатор, используемый для смешивания. Из гистограммы можем узнать, что в среднем используемо количество газа будет 10, медиана, наверняка, такая же. Кроме того мы можем увидеть наличие выбросов. Будем держать его в уме, но ближе их изучим (и при необходимости избавимся), возможно, уже после объединения таблиц. 

В теории мы можем проверить - посчитать корреляцию по ключам количества газа и суммарного расхода мощности, чтобы это проверить. Но для начала продолжим изучать таблицы, а сюда вернемся позже. 

In [ ]:
try:
    temp = pd.read_csv('/Users/BrightFuture/Desktop/Projects/Final project industry/data/data_temp_new.csv')
except FileNotFoundError:
    print('Error: File not found. Please provide the correct file path.')

In [ ]:
display(temp)

Визуально есть ощущение, что температуру замеряют перед каждым нагреванием, но не сразу после него/ в процессе. То есть если я правильно понимаю, греют они примерно на глаз. Интерсно, а как они определяют, сколько надо греть и/или до какой температуры?  

Вообще, если честно, я не особо верю, что за 5 минут рабочие способны взять пробы и провести полный химический анализ. Похоже, нам немного преукрашивают ситуацию. 

In [ ]:
try:
    wire = pd.read_csv('/Users/BrightFuture/Desktop/Projects/Final project industry/data/data_wire_new.csv')
except FileNotFoundError:
    print('Error: File not found. Please provide the correct file path.')

In [ ]:
display(wire)

In [ ]:
wire.hist(figsize=(12,12))

И снова нам тут особо ничего такого, что могло бы помочь в создании модели, не узнать.

In [ ]:
try:
    wire_time = pd.read_csv('/Users/BrightFuture/Desktop/Projects/Final project industry/data/data_wire_time_new.csv')
except FileNotFoundError:
    print('Error: File not found. Please provide the correct file path.')

In [ ]:
display(wire_time)

Я не до конца понимаю технологический процесс влияния проволоки, и не готов сейчас слишком глубоко туда погружаться. На самом деле мне досататочно того, что применений действительно много и того, что я уверен, что мой алгоритм сам посчитает все достаточно точно и отберет наиболее важные параметры. 

Что бы я отметил - так это тот факт, что объектов меньше,чем ключей, так что не для кажого продукта нужна проволока, в отличие от легирующих добавок. 

In [ ]:
print(arc[arc['key']==269])
print(temp[temp['key']==269])
print(gas[gas['key']==269])
print(bulk_time[bulk_time['key']==269])
print(wire_time[wire_time['key']==269])

In [ ]:
print(arc[arc['key']==270])
print(temp[temp['key']==270])
print(gas[gas['key']==270])
print(bulk_time[bulk_time['key']==270])
print(wire_time[wire_time['key']==270])

# EDA

Теперь, когда мы посмотрели все таблицы и уже представляем себе структуру данных, можно с ними немного поработать: 

1)Визуализировать данные и/или рассчитать какие-то аггрегированные показатели.

2)Преобразовать, как мы задумывали - вычисления показателей и аггрегированные таблицы, возможно, новые графики

И есть еще одна маленькая, но важная история, которая нам портит не только модель, но и исследовательский анализ данных. Мы уже знаем, что есть один феноменальный выброс в "реактивной мощности" и разобраться с ним надо сразу, пока он не наследил.

In [ ]:
print(arc[arc['Реактивная мощность']<0])

А вот и наш выброс. Это даже не выброс, а просто ошибка. Она настолько огромна, что уничтожает все корреляции со столюцом 'Реактивная мощность'

Думаю, никого не обидит, что я удалю эту партию целиком. Хотя в теории я мог бы рассчитать корреляцию между активной и реактивной мощностью, доказать, что она высокая и потом по коэффициенту рассчитать реактивную мощность для этого конкретного подогрева. 

In [ ]:
arc = arc.drop(arc[arc['key']==2116].index)

In [ ]:
print(arc[arc['Реактивная мощность']<0])

In [ ]:
ener = arc[['key','Активная мощность', 'Реактивная мощность']]

In [ ]:
print(ener)

In [ ]:
ener.describe()

In [ ]:
print(ener[ener['Реактивная мощность']<0])

In [ ]:
ener = ener.drop(ener[ener['Реактивная мощность']<0].index)

In [ ]:
ener2 = ener.groupby('key')[['Активная мощность', 'Реактивная мощность']].sum()

In [ ]:
print(ener2)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(ener2.index, ener2['Активная мощность'], color='blue', linewidth=2, label='Активная мощность')
plt.plot(ener2.index, ener2['Реактивная мощность'], color='red', linewidth=2, label='Реактивная мощность')
plt.xlabel('№ of process')
plt.ylabel('Energy consumption')
plt.title('Consumption of energy for steel production')
plt.grid(True)
plt.legend()
plt.show()

Мы видим колебания активной мощности и пока не знаем, чем они вызваны. Я бы проверил корреляцию с затраченным временем на нагрев и с "Газом"

Что-то есть, но если бы газ превращался в электричество, думаю, корреляция была бы значительно ближе к 1. Хотя я только что обнаружил различное количество рядов, придется пересчитать

In [ ]:
merged_df = ener2.merge(gas, left_index=True, right_on='key')

In [ ]:
print(merged_df['Активная мощность'].corr(merged_df['Газ 1']))

In [ ]:
arc2 = arc.copy()

In [ ]:
arc2['Начало нагрева дугой'] = pd.to_datetime(arc2['Начало нагрева дугой'])

In [ ]:
arc2['Конец нагрева дугой'] = pd.to_datetime(arc2['Конец нагрева дугой'])

In [ ]:
arc2.info()

In [ ]:
arc2['time_heated'] = arc2['Конец нагрева дугой']-arc2['Начало нагрева дугой']

In [ ]:
arc2['time_heated_sec'] = arc2['time_heated'].dt.total_seconds()

In [ ]:
print(arc2)

In [ ]:
print(arc2['Активная мощность'].corr(arc2['time_heated_sec']))

In [ ]:
arc2['total_power'] = (arc2['Активная мощность']**2 + arc2['Реактивная мощность']**2)**0.5

In [ ]:
print(arc2['total_power'].corr(arc2['time_heated_sec']))

In [ ]:
print(arc2['Активная мощность'].corr(arc2['Реактивная мощность']))

Поскольку корреляции между мощностью и временем отсутствует, можно сделать вывод, что мощность представляет расход энергии в единицу времени. Заначит, нам придется считать производный показатель и здесь. Хорошо, что теперь мы это знаем. 

В выводе хочется отметить, что мы двольно детально изучили данные и теперь полностью понимаем как двигаться дальше и готовы к этому. 

Дальнейший план работы:

1) Доделать предобработку данных. Убрать все данные, которые не подходят для целей нашего прогнозирования.

2) Создание дополнительных параметров. У меня ощущение, что того, что у нас есть на данный момент будет недостаточно. 

3) Агрегирование всех данных в общую таблицу для созадния модели машинного обучения.

4) Создание 3 моделей машинного обучения. Предварительно планируются модель регрессии, модель случайного леса и модель градитентного бустина XGboost, но если в какой-то момент появится желание создать любую другую модель, я не буду отказывать себе в удовольствии. (пока не забыл - я бы попробовал различные виды регуляризации в регрессии, чтобы проверить мое предположение о чрезмерном влиянии энергии затраченной на подогрев на температуру и попробовать выявить и другие важные факторы)

5) Выбор лучшей модели по показателю MAE на кросс-валидации. Тестирование модели и вывод.

# Предобработка данных

## Время нагревания

In [ ]:
arc2['total_energy'] = arc2['total_power'] * arc2['time_heated_sec']

In [ ]:
arc_attn = arc2[['key', 'time_heated_sec', 'total_power', 'total_energy']]

In [ ]:
arc_grouped = arc_attn.groupby('key')[['time_heated_sec', 'total_power', 'total_energy']].sum()

In [ ]:
print(arc_grouped)

Вот так, я думаю, должна выглядеть финальная таблица признаков, которые можно получить из их информации о нагревании стали.

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(arc_grouped['time_heated_sec'], bins=20, color='crimson', edgecolor='black', alpha=0.7)

plt.xlabel('Время')
plt.ylabel('Частота')
plt.title('Обзор Времени нагрева на одну партию')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
arc_grouped['time_heated_sec'].describe()

In [ ]:
start_times = arc2.groupby('key')['Начало нагрева дугой'].first()
end_times = arc2.groupby('key')['Конец нагрева дугой'].last()
arc_times = pd.concat([start_times, end_times], axis=1)
print(arc_times)

In [ ]:
print(arc_times.info())

## Замеры температуры

In [ ]:
display(temp)

In [ ]:
temp.info()

In [ ]:
temp_nuns = temp[temp['Температура'].isna()]
print(temp_nuns['key'].unique())

А нулей у нас тут много... по сути все ключи после 2499 без температуры. Но странно, что общая доля всех пропусков 23%, а общая доля уникальных значений 30%.

In [ ]:
exp = temp[temp['key'].isin(temp_nuns['key'].unique())]
print(exp.head(30))

In [ ]:
counts = exp.groupby('key')['Температура'].count().sort_values(ascending=False)
print(counts)

Теперь все стало на свои места. У всех партий, начиная с 2500 есть только одна температура - и это стартовая темпаратура. Идеально было бы, если бы нам дали ее в конце, чтобы мы протестировали данные. А то бить 2499 партий на 3 выборки это больно.

Оказалось, это неизбежно и новых данных не предвидится. Так что пока держим в уме, что все данные партии, которые придется исключить из-за отсутствия целевой температуры, находятся  в temp_nuns['key'].unique()

In [ ]:
start_temp = temp.groupby('key')['Температура'].first()
end_temp = temp.groupby('key')['Температура'].last()
temperatures = pd.concat([start_temp, end_temp], axis=1)
temperatures.columns = ['start_temp', 'end_temp']
print(temperatures)

Вот в таком виде нам нужны данные для нашей модели.Начальная температура - это важный параметр, а конечная температура - целевой показатель. Осталось только убрать объекты с отсутсвтвующим целевым показателем.

Таблицы gas, bulk и wire мы оставим как есть, поскольку они уже представляют собой признаки. 

таблицы bulk_time и wire_time мы пока трогать не будем. В теории они нужны, чтобы проверить данные на валидность, однако на данный момент я не буду этим заниматься по 2 причинам:

1) я хочу сначала посторить хотя бы одну модель и проверить, насколько у нас все плохо или хорошо, чтобы оценить наши данные. 

2) я не представляю, как автоматизировать этот процесс. Пока что у меня есть идея только сделать общую таблицу (из arc, bulk_time и wire_timeб temp) и отсортировать ее по таймлану. Но вот как найти ошибки - это уже вопрос. Снова сказываются 50 часов, потраченные на SQl... там есть функция SELECT ROW_NUMBER() OVER () и при помощи нее я бы проранжировал все операции в рамках одной партии (ключа) Но потом бы пришлось еще каким-то образом выяснить, где есть какие-то операции, сделанные позже, чем финальная температура, а вот тут у меня пока идей нет. Буду признательным за подсказку, если эта часть обязательная.


## Время между замерами

Для начала я хочу попробовать создать и проанализировать дополнительный параметр - время между первым и последним замерами температуры. Есть идея (которая принесла нам прмерно 0.2 MAE), что партии обрабатываются с разной скоростью, а некоторые партии вообще простаивают по причнам пересменок, перерывов и т.д.

И если немного развить идею, получится, что чем дольше время между первым и последним замерами, тем больше сталь успеет остыть.

In [ ]:
start_temp_time = temp.groupby('key')['Время замера'].first()
end_temp_time = temp.groupby('key')['Время замера'].last()
temperatures_time = pd.concat([start_temp_time, end_temp_time], axis=1)
temperatures_time.columns = ['first_measure', 'last_measure']
print(temperatures_time)

In [ ]:
temperatures_time['first_measure'] = pd.to_datetime(temperatures_time['first_measure'])
temperatures_time['last_measure'] = pd.to_datetime(temperatures_time['last_measure'])

In [ ]:
temperatures_time['total_time'] = temperatures_time['last_measure'] - temperatures_time['first_measure']
temperatures_time['total_time'] = temperatures_time['total_time'].dt.total_seconds()

In [ ]:
print(temperatures_time)

In [ ]:
temperatures_time = temperatures_time.drop(['first_measure', 'last_measure'], axis=1)

In [ ]:
print(temperatures_time)

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(temperatures_time, bins=20, color='limegreen', edgecolor='black', alpha=0.7)

plt.xlabel('Время')
plt.ylabel('Частота')
plt.title('Обзор Времени затрачиваемого на одну партию')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

Ну вот и получается, что среднее время нагревания у нас около 1000 секунд (15 мин), среднее время производтва около 2000 секунд (30 мин), при этом есть продукты, которые 6 часов "в производтве" а по факту, навернякак ночуют... Да и трехчасовые тоже присутствуют. В общем, параметр сильный, не зря он нам так сильно снизил MAE.

Прикол в том, что  хоть показателей температуры и нет в таблице, время замеров есть. Нам это не помешает, потму что у нас невалидные данные сами отсеятся при слиянии таблиц но сам факт, конечно смешной.

И добавлю, что в идеале я бы поубирал все эти загулы, но у нас тогда, наверное, вообще очень мало данных останется. 

# Дообработка данных

## Температура плавления

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(temp['Температура'], bins=20, color='orange', edgecolor='black', alpha=0.7)

plt.xlabel('температура')
plt.ylabel('Частота')
plt.title('Обзор замеров температур')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

Видимо, у нас есть выбросы и они довольно серьезные. Хочу отметить, что температура плавления стали начинается от 1350 градусов и это для самых легковыплавляемых сплавов. Значит, все замеры/партии, где температура ниже этой границы - выбросы. 

In [ ]:
print(temp[temp['Температура']<1350])

In [ ]:
print(temp[temp['key'].isin([867, 1214,1619, 2052, 2561])])

In [ ]:
print(arc[arc['key']==867])

Да, полный бред. Даже если бы я в теории поверил, что в ковш засыпали какие-нибуль металические штуки, чтобы расплавить, что противоречит описанию технологического процесса, невозможно за 3 минутки 100 тонн металла расплавить и нагреть на 400 градусов. 

Поздравляю, мы нашли еще немного лютых выбросов. 

In [ ]:
invalid_keys = [867, 1214,1619, 2052, 2561]

И температура плавления приносит грифиндору 0.2 градуса на валидационной выборке и 0.6 градуса на тестовой выборке.

## Время остывания

In [ ]:
check_tab= temperatures.merge(arc_grouped, left_index=True, right_index=True)

In [ ]:
check_tab = check_tab.merge(temperatures_time, left_index=True, right_index=True)

In [ ]:
display(check_tab)

Как насчет того, чтобы найти коэфициент, насколько общее время обрабоки (по факту от начального замера до конечного) дольше, чем время нагревания. И на этом пространстве мы тоже найдем выбросы. 

In [ ]:
check_tab['prod_heat_rate'] = check_tab['total_time']/check_tab['time_heated_sec']

In [ ]:
display(check_tab)

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(check_tab['prod_heat_rate'], bins=300, color='dodgerblue', edgecolor='black', alpha=0.7)

plt.xlabel('коэффицент длительности остывания')
plt.ylabel('Частота')
plt.title('Отношение длительности остывания к длительности нагрева')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

Ничего такого, чего мы не ожидали. А почему ожидали? - да потому, что знали про пересменки и ночевки... Вопрос в другом, как нам лучше определить границу, после которой данные будут считаться выбросами. Если честно, я бы воспользовался правилом 3 сигм, чтобы опираться на какую-нибудь статистику, нежели на "экспертное мнение"

Но если подумать, то это немного неправильно. У нас тут распределение со скосом вправо, а в таких распределениях находятся несколько иначе. Предлагаю воспользоваться межквартальным размахом. 

IQR = Q3-Q1

Верхняя граница выбросов = Q3 + 3 * IQR

In [ ]:
print(check_tab['prod_heat_rate'].describe())

In [ ]:
q1 = 1.987030
q3 = 3.538012

In [ ]:
iqr = q3 - q1

In [ ]:
limit = q3 + 3*iqr
print(limit)

так-то это уж как-то совсем мало и значительно меньше, чем мне казалось, вручную я бы ни за что меньше 10 не поставил. С другой стороны кто я такой, чтобы спорить со статистикой (я и так увеличил кэф IQR с 1,5 до 3, что, конечно, же допустимо), так что раз уже решил делать по уму, то так тому и быть. 

In [ ]:
print(check_tab[check_tab['prod_heat_rate']>limit])

еще 95 выбросов. Часть из них, конечно, придестя на уже отброшенные значения. Но вообще интересно, что же у нас останется. В результате самых элементарных преобразований оставалось 2324, вроде

In [ ]:
invalids2 = check_tab[check_tab['prod_heat_rate']>limit].copy()

In [ ]:
invalid_keys2 = invalids2.index

In [ ]:
print(invalid_keys2)

 # Создание агрегированной таблицы признаков

In [ ]:
final_tab = temperatures[~temperatures.index.isin(temp_nuns['key'])].copy()

In [ ]:
final_tab = final_tab[~final_tab.index.isin(invalid_keys)]

In [ ]:
final_tab = final_tab[~final_tab.index.isin(invalid_keys2)]

In [ ]:
print(final_tab)

In [ ]:
try:
    print(final_tab.loc[867])
except KeyError:
    print("нет такого ключа")

In [ ]:
final_tab = final_tab.merge(arc_grouped, left_index=True, right_index=True)

In [ ]:
print(final_tab)

In [ ]:
final_tab = final_tab.merge(gas, left_index=True, right_on='key')

In [ ]:
display(final_tab)

Вот тут у нас случился интересный момент, который хочу отметить (поскольку пришлось фиксить) - с последнем мерджем у нас дропнулись индексы и теперь мы уже объединяем все таблицы по колонке 'key'

In [ ]:
final_tab = final_tab.merge(bulk, left_on='key', right_on='key')

In [ ]:
display(final_tab)

In [ ]:
final_tab = final_tab.merge(wire, left_on='key', right_on='key')

In [ ]:
display(final_tab)

In [ ]:
final_tab = final_tab.merge(temperatures_time, left_on='key', right_index=True)
display(final_tab)

In [ ]:
final_tab.set_index('key', inplace=True)

In [ ]:
display(final_tab)

In [ ]:
final_tab['cooling_time'] = final_tab['total_time'] - final_tab['time_heated_sec']
final_tab = final_tab.drop('total_time', axis=1)

In [ ]:
final_tab.fillna(0, inplace=True)

In [ ]:
display(final_tab)

У нас получилась отличная, на мой взгляд, таблица признаков, давайте же скорее посмотрим, как модель сможет по нашим признакам рассчитать температуру.

In [ ]:
correlation_matrix = final_tab.corr()

In [ ]:
f, ax = plt.subplots(figsize=(25, 20))
cmap = sns.diverging_palette(170, 130, as_cmap=True)
ax = sns.heatmap(correlation_matrix, cmap=cmap, annot=True, vmax=1, vmin=0, center=0, linewidths=0, xticklabels=True, yticklabels=True)
plt.title('heatmap - матрица корреляций', fontsize=25)
plt.show()


Если изучить кореляцию  между признаками, у нас есть 3 очень сильно коррелирующих параметра: время нагрева, общая мощность, всего энергозатраты. Оно и понятно, поскольку энергозатраты = время * мощность. Тем не менее все корреляции внутри нашей итоговой таблицы меньше 0.9, поэтому удалять мы ничего не будем. 

В каком-то смысле у нас есть мультиколинеарность Шредингера - в зависимости от того, какой трэшхолд поставить, она либо есть либо нет) Однако сейчас, после того как был озвучен порог в 0.9, подгонять трэшхолд под результат, чтобы что-то удалить, было бы уже неправильно.

Еще один вопрос здесь в другом - у нас есть факторы, которые очень слабо коррелируют с целевым показателем. Да, зависимости бывают и нелинейными, и на основе одних коэффициентов корреляции удалять признаки мы не будем (к тому же модель все сама откалибрует), но хотя бы присмотреться стоит. В основном это значения большинства показателй bulk и wire.

In [ ]:
final_tab = final_tab.drop(['Wire 3', 'Bulk 10', 'Bulk 5', 'Wire 6', 'Bulk 2', 'Wire 9', 'Bulk 9',
       'Bulk 13', 'Wire 8', 'Wire 5', 'Bulk 8'], axis=1)

In [ ]:
display(final_tab)

# Создание моделей

## Регрессия

In [ ]:
y = final_tab['end_temp']
x = final_tab.drop('end_temp', axis=1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
print(x_train.shape, y_train.shape)

теперь давайте наконец-то проверим модель линейной регрессии. 

In [ ]:
model = LinearRegression()

scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_absolute_error', cv=5)
mae_scores = -scores
average_mae = mae_scores.mean()

print("Cross-Validation MAE scores:", mae_scores)
print("Average MAE:", average_mae)

Давай взглянем на коэффициенты

In [ ]:
model.fit(x_train, y_train)
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

In [ ]:
columns = x.columns

In [ ]:
coeffs = pd.DataFrame({'feats': columns, 'coef': model.coef_})
print(coeffs.sort_values(by = 'coef', ascending = False))

И тут настал момент, когда я прозрел и понял, что коэффициенты ни чета важности факторов, поскольку они не определяют саму важность факторов, а также коэффициент зависит от значений параметров, что делает сортировку по коэффициенту по сути бесполезной. Так что нам остается только выяснять все на древесных моделях. 

Кстати, у нашего злополучного Wire 5 коэффициент аж 1.676437e-14 :)

## Случайный лес

In [ ]:
%%time
param_grid = {
    'n_estimators': [30, 50, 100, 200],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 8, 10],
    'min_samples_leaf': [1, 2, 4, 5]
}

rf_model = RandomForestRegressor()

random_search = RandomizedSearchCV(rf_model, param_distributions=param_grid, n_iter=10,
                                   cv=5, scoring='neg_mean_absolute_error')

random_search.fit(x_train, y_train)

best_model = random_search.best_estimator_
best_params = random_search.best_params_

print(best_params)

In [ ]:
cv_scores = cross_val_score(best_model, x_train, y_train, cv=5, scoring='neg_mean_absolute_error')
cv_scores = -cv_scores
mean_cv_score = cv_scores.mean()
print(cv_scores)
print("Average MAE:", mean_cv_score)

In [ ]:
rf_model = RandomForestRegressor(n_estimators= 200, min_samples_split= 2,
                                 min_samples_leaf= 2, max_depth= 15)
rf_model.fit(x_train, y_train)

In [ ]:
importances = rf_model.feature_importances_

features = x.columns
for feature, importance in zip(features, importances):
    print(feature, importance)

In [ ]:
sorted_indices = np.argsort(importances)[::-1]
sorted_importances = importances[sorted_indices]

cumulative_importances = np.cumsum(sorted_importances)

In [ ]:
print(sorted_indices)

In [ ]:
print(sorted_importances)

In [ ]:
print(cumulative_importances)

In [ ]:
plt.plot(range(len(sorted_importances)), cumulative_importances)
plt.xlabel('Number of Features')
plt.ylabel('Cumulative Importance')
plt.title('Cumulative Feature Importances')
plt.grid(True)
plt.show()

Первые по важности 19/20 признаков дают 99.1%/99.5% важности всех факторов в совокупности. Так что по сути, можно убрать 10-11 значений, ничего по сути не поменятеся, как я считаю. 

А вот и 11 самых бесполоезных значений. Следует ли их удалить!?

Если мы удалим неважные фичи, как мы будем объяснять заказчику, что они неважные?) 

случайные лес дает качество хуже, чем регрессия, хотя оно тоже довольно неплохое и проходное. Теперь проверим модель lightgbm, в нее я верю еще больше.

## Модель LightGBM

In [ ]:
feature_names = final_tab.columns

In [ ]:
print(feature_names)

In [ ]:
train_data = lgb.Dataset(x_train, label=y_train)
test_data = lgb.Dataset(x_test, label=y_test)

In [ ]:
display(test_data)

In [ ]:
param_grid2 = {
    'boosting_type': ['gbdt', 'dart', 'goss'],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 5, 7, 9],
    'num_leaves': [16, 31, 63, 127],
    'num_boost_round': [100, 300, 500, 1000],
    'early_stopping_rounds': [20, 50, 100]
}

In [ ]:
%%time
lgb_model = lgb.LGBMRegressor()

random_search2 = RandomizedSearchCV(lgb_model, param_distributions=param_grid, n_iter=10, scoring='neg_mean_absolute_error', cv=5)

random_search2.fit(x_train, y_train)

best_model2 = random_search2.best_estimator_
best_score2 = random_search2.best_score_
best_params2 = random_search2.best_params_
print(best_score2)
print(best_params2)

In [ ]:
cv_scores = cross_val_score(best_model2, x_train, y_train, scoring='neg_mean_absolute_error', cv=5)
mean_cv_score = cv_scores.mean()
print(cv_scores)
print(mean_cv_score)

## Маленькая придумка

Да, качество еще лучше. Можем ли мы еще улучшить качество? У меня 2 идеи: поискать выбросы, потому что мы временно пропустили этот процесс и попробовать сделать свой бустинг и первым слоем сделать в нем регрессию, потому что у нас отличные линейные связи и попробовать эту регрессию немнгожечко пробустить. 

In [ ]:
base_model = LinearRegression()
base_predictions = cross_val_predict(base_model, x_train, y_train, cv=5)
base_mae = mean_absolute_error(y_train, base_predictions)

boosting_model = GradientBoostingRegressor(loss='absolute_error', n_estimators=100, learning_rate=0.1)
residuals = y_train - base_predictions
boosted_predictions = cross_val_predict(boosting_model, x_train, residuals, cv=5)
boosted_predictions = base_predictions + boosted_predictions
boosted_mae = mean_absolute_error(y_train, boosted_predictions)

print("Base Model MAE:", base_mae)
print("Boosted Model MAE:", boosted_mae)

Так мне больше нравится! На данный момент это лучшая модель. А теперь, когда мы убедились, что данные у нас адекватные и качественную модель построить можно и легко, я хочу вернуться в прошлое и прогнать наши данные и просеять их через сито времени подготовки. Или придумать альтернативный параметр. 

# Тестирование

Есть 2 стула : на одном тестирование на уже обученной модели, на другом надо объединить тренировочную и валидационную выборки, обучить на них новую модель с заданными гиперпараметрами и потом протестировать. (У обоих есть свои преимущества и свои недостатки, при чем зеркальные относительно друг друга) И парадокс в том, что куда бы я ни сел, меня обязательно просят пересесть. Поэтому я просто протестирую уже обученную модель, а потом пересяду, когда попросишь, потому что лично мне больше нравится обучать тестовую модель на объединенной выборке, т.к. у нас и без того данных мало. 

In [ ]:
base_model = LinearRegression()
base_model.fit(x_train, y_train)
base_predictions2 = base_model.predict(x_test)
base_mae2 = mean_absolute_error(y_test, base_predictions2)

boosting_model2 = GradientBoostingRegressor(loss='absolute_error', n_estimators=100, learning_rate=0.1)
residuals2 = y_train - base_model.predict(x_train)
boosting_model2.fit(x_train, residuals2)
boosted_predictions2 = base_predictions2 + boosting_model2.predict(x_test)

boosted_mae2 = mean_absolute_error(y_test, boosted_predictions2)

print("Base Model MAE:", base_mae2)
print("Boosted Model MAE:", boosted_mae2)

Хочу обратить внимание, что хотя модель регрессии изначально дала низкий результат, как непроходной по заданию, так и намного ниже, чем на валидационной выборке. Настолько большой разницы для обычной регрессии я и правда не ожидал. Объяснить все это могу только ассиметричностью распределения данных и наличием выбросов, поскольку регрессия никак не связана с валидационной выборкой в процессе обучения. 

  # Вывод

Пока выводов немного, мне не очень понравился итоговый результат, даже если он и проходной, после такого результата на валидацинной выборке. Возможно, нам надо еще почитстить данные, например, убрать слишком долго остывающие партии... как вообще может сталь не застыть, если ее на ночь оставить? Возможно, есть какие-то температурные маркеры, придется это тоже изучить. Если менять random_state, может получиться совсем иная картина, но мы будем работать с тем, что имеем. Да и стоит один раз удалить несколько выбросов и выборки изменятся в корне. 

В общем я очень надеюсь на то, что ты посмотришь мою работу, укажешь мои ошибки и подскажешь, как улучшить мой проект так, чтобы он был достоин звания финального или выпускного.

## V 2.0

А вот теперь ситуация перевернулась с ног на голову. До удаления очередной партии выбросов ( раздел доработка данных) у меня был МАЕ на валидационной выборке 5.5, а на тестовой 6.55. После переработки занчение на тестовой стало 5.64, что уже не стыдно. В общем, переподгружу тетрадку, надеюсь, теперь мой проект произведет чуть более приятное впечатление. Тем не менее я еще в поисках зон роста и, надеюсь, ты мне что-нибудь еще подскажешь :)

## V 3.0

На этот раз я уже совсем не обеспокоен тем, что MAE на тестовой выборке повысился, потому что это просто дело выборки. Данные все еще очень кривые, даже несмотря на то, сколько мы удалили (я бы удалил больше и обосновал почему в пункте 6.2). В подтверждение моих слов работает разброс на кросс-валидациях, который достигает полутора пунктов (примерно 25%).

# Отчёт

## Выполнение плана

План был выполнен в полном объеме. Мы проанализировали данные под разными углами, нашли очень много неподходящих данных для нашей модели и выбросов. Затем мы создали общую таблицу признаков и целевого показателся, чтобы на основе него построить модели. Я не стал эксперементировать с регуляризаций в регрессии, зато создал на одну модель больше, чем планировал. Не стал отдельно исследовать регуляризацию, потому что после встречи в зуме я понял, что наша задача выглядит иначе, чем я представлял. 

## Трудности

В ходе проекта мне пришлось столкнуться с рядом трудностей. 

1)Пришлось осознать, как устроен процесс пороизводства и что от нас не требуется ничего связанного с экономией, только элементарное предсказание температуры. 

2)Мне не понравилось, как хранятся данные в таблицах bulk_time и wire_time, это затруднило (и, наверное, в моем случае загубило) создание единого таймлайна, по которому можно отследить процесс целиком. 

3)Трудно было определить, как поступить с данными, где время ожидания (остывания) слишком большое. Проблема, которую я тут вижу, что эти данные для нашей модели являются выбросами, потому что мы в каком-то роде прогнозируем измениения температуры в связи с нагреваними (но не остыванием). И это с одной стороны не очень хорошо, т.к. такие неэффективные процессы на нашем заводе случаются довольно регулярно, при этом у нас недостаточно данных, чтобы точнее научиться прогнозировать эти неффективные процессы. 

4)Для меня было непривычно оценивать важность факторов разными, я раньше этого не делал. 

5)Из-за большого разброса в данных качетво модели ощутимо скачет, что немного меня беспокоило, когда я получал более низкое качество модели. В итоге я начинал проверять другие random_state, чтобы успокоить себя и понять, что я делаю все правильно глобально, а качество ухудшилось потому, что какие-то объекты были удалены и это изменило тренировочную и валидационную выборки. Также помог потом переход на оценку моделей кросс-валидацией вместо валидационной выборки. 

Отвечая на вопрос, как я преодолевал эти трудности, ответ в каждом случае один и тот же - я тратил время на раешение возникшей проблемы. И чем больше тратил времени, тем лучше было решение.

## Ключевые шаги

1)Понимание процесса и конечной цели.

2)Хороший и разносторонний анализ данных. Понимание не только того, какие даныне у нас есть, но и то, что они подразумевают, как их можно трансформировать, где могут быть проблемы, выбросы, слабые места. 

3)Важно было бы с самого начала настроить оценку модели через кросс-валидацию, чтобы получать более сблансированную оценку и более точную обратную связь при дообработке данных.

## Итоговая модель

Моя итоговая модель - самодельный бустинг, который начинается с линейной регрессии, а потом ошибки прогнозируются при помощи GradientBoostingRegressor из Sklearn. Сделано это было изначально ради интереса. Хороших результатов я ожидал, потому что линейная регрессия сама по себе дала очень хороший результат по сравнению с "древесными" моделями. Итоговая метрика MAE на тестовой выборке равна 5.896. Показатель мог быть выше, но, как я уже объяснял, из-за разброса в данных и небольшого количетва этих самых данных итоговый показатель будет варьироваться примерно от 5.5 до 6.2 в зависимости от изначально заданного RANDOM_STATE и/или изменения выборки любым другим способом.

## Параметры

Парметры, которые я создал и стали основными: стартовая температура, время нагревания, время остывания, общие энергозатраты. Они и еще те параметры, которые были изначально, а именно легирующие добавки, газ  и провода участвуют в модели предсказааний. При этом 9-10 параметров добавок и проволоки мы можем убрать без потери качества. Также один параметр можно удалять как из-за мультиколинеарности, так и из-за бесполезности на выбор.

Что касается предобработки по созданным признакам, были убраны все партии, где температура ниже температуры плавления стали. Также были удалены все партии со слишком высоким коэффиицентом соотношения общего времени производства ко времени нагревания, так как это означает, что продукт слишком долго остывал в котле по несвязанным с производством причинам. 

##  Гиперпараметры

Гиперпараметров у регрессии нет, но есть у бустинга, при помщи которого мы предсказываем ошибки, они следующие : loss='absolute_error', n_estimators=100, learning_rate=0.1. У нас не было необходимости слишком сильно бустить модель на довольно скудных данных, чтобы избежать переобучения. 

## Улучшение модели

Улучшить модель можно с помощью:

1) увеличения количества данных. 

2) Получения более детальных отчетов по конкертным партиям от заказчика - нам надо лучше разобраться со слишком долгими остываниями. 

3) В теории можно было бы попробовать научиться предсказывать температуру за меньший период, чем партия, например, за каждое отдельное нагревание, остывание, перемешивание и т.д. для этого нам бы, возможно, понадобилось больше замеров температуры. Но это долгий и сложный процес, который может и не дать желаемого результата. Вопрос, есть ли у нас на это время и ресурсы. 